<a href="https://colab.research.google.com/github/MatteoOnger/algo-collusion-mm/blob/main/notebooks/notebook_exp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithmic Collusion in Market Making - Offline Game Analysis

A notebook to find/check equilibria in the Glosten-Milgrom model.

## Notebook Initialization

### Colab Environment Setup

In [ ]:
# Do NOT run this cell in local environment - it's intended for Google Colab only.

# Clone GitHub repository
!git clone https://github.com/MatteoOnger/algo-collusion-mm.git

# Set working directory
%cd /content/algo-collusion-mm

# Install dependencies
!pip install --quiet .

### Local Environment Setup

In [6]:
# Do NOT run this cell in Google Colab - it's intended for local Jupyter Notebooks only.

# Autoreload imports
%load_ext autoreload
%autoreload 2

# Select interactive backend for matplotlib
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Offline Game Analysis

In [7]:
import itertools
import numpy as np
import time

import algo_collusion_mm.utils.gtu as gtu

from datetime import datetime


### Find all NEs and CCEs

In [ ]:
prices =  np.round(np.arange(0.0, 1.0 + 0.2, 0.2), 2)
action_space = np.array([(ask, bid) for ask in prices for bid in prices if (ask  > bid)])

n_makers = 2
action_spaces = np.repeat(action_space[None, :], repeats=n_makers, axis=0)

print(f'Action spaces shape: {action_spaces.shape}')

joint_action_space, rewards = gtu.compute_joint_actions_and_rewards(action_spaces, true_value=0.5, tie_breaker='rand')

print(f'Joint action space shape: {joint_action_space.shape}')
print(f'Rewards shape: {rewards.shape}')
print('--------------------------------------------')

scaled_rewards = rewards

start_time = time.time()
print('Search pure CCEs:')

for a in itertools.product(*[range(s) for s in rewards.shape[:-1]]):
    prof = np.zeros(rewards.shape[:-1])
    prof[a] = 1.0
    if gtu.is_cce(scaled_rewards, prof):
        print(f"- {a} -> {str(joint_action_space[a].swapaxes(-1, -2)).replace('\n', '')} is a CCE")

end_time = time.time()
execution_time = end_time - start_time
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f'Done at {current_time} | Execution time: {execution_time:.2f} seconds')
print('--------------------------------------------')

start_time = time.time()
print('Search pure NEs:')

for a in itertools.product(*[range(s) for s in rewards.shape[:-1]]):
    prof = np.zeros((n_makers, len(action_space)))
    prof[np.arange(n_makers), a] = 1.0
    if gtu.is_ne(scaled_rewards, prof):
        print(f"- {a} -> {str(joint_action_space[a].swapaxes(-1, -2)).replace('\n', '')} is a NE")

end_time = time.time()
execution_time = end_time - start_time
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f'Done at {current_time} | Execution time: {execution_time:.2f} seconds')
print('--------------------------------------------')

### Find best CCE

In [ ]:
prices =  np.round(np.arange(0.0, 1.0 + 0.2, 0.2), 2)
action_space = np.array([(ask, bid) for ask in prices for bid in prices if (ask  > bid)])

n_makers = 2
action_spaces = np.repeat(action_space[None, :], repeats=n_makers, axis=0)

joint_action_space, rewards = gtu.compute_joint_actions_and_rewards(action_spaces, true_value=0.5, tie_breaker='rand')
scaled_rewards = rewards

prof, reward = gtu.find_best_cce(rewards, objective='social_welfare', solver='SCIPY')

print(f'Search best CCE:')
print(f'- Reward: {reward}')
print(f'- Profile: {np.where(prof > 1e-05)}')
print('--------------------------------------------')

### Check NE and/or CCE

In [ ]:
prices =  np.round(np.arange(0.0, 1.0 + 0.2, 0.2), 2)
action_space = np.array([(ask, bid) for ask in prices for bid in prices if (ask  > bid)])

n_makers = 2
action_spaces = np.repeat(action_space[None, :], repeats=n_makers, axis=0)

print('Action space:')
for i in range(len(action_space)):
    print(f' - {i} -> {action_space[i]}')

joint_action_space, rewards = gtu.compute_joint_actions_and_rewards(action_spaces, true_value=0.5, tie_breaker='rand')

print(f'Joint action space shape: {joint_action_space.shape}')
print(f'Rewards shape: {rewards.shape}')
print('--------------------------------------------')

scaled_rewards = rewards

print('Check CCE:')
prof = np.zeros(rewards.shape[:-1])
prof[(7, 7)] = .5
prof[(5, 5)] = .3
prof[(7, 5)] = .1
prof[(5, 3)] = .1
print(f'Profile sum: {prof.sum()}')
print(f'Expected reward: {(rewards * prof[:,:, None]).sum(axis=tuple((i for i in range(n_makers))))}')
print(f' --> {gtu.is_cce(scaled_rewards, prof, verbose=True)}')
print('--------------------------------------------')

In [ ]:
prices =  np.round(np.arange(0.0, 1.0 + 0.2, 0.2), 2)
action_space = np.array([(ask, bid) for ask in prices for bid in prices if (ask  > bid)])

n_makers = 2
action_spaces = np.repeat(action_space[None, :], repeats=n_makers, axis=0)

print('Action space:')
for i in range(len(action_space)):
    print(f' - {i} -> {action_space[i]}')

joint_action_space, rewards = gtu.compute_joint_actions_and_rewards(action_spaces, true_value=0.5, tie_breaker='rand')

print(f'Joint action space shape: {joint_action_space.shape}')
print(f'Rewards shape: {rewards.shape}')
print('--------------------------------------------')

scaled_rewards = rewards

print('Check NE:')
prof = np.zeros((n_makers, len(action_space)))
prof[0, 4] = .68
prof[1, 5] = 1 - .68
prof[0, 4] = .8
prof[1, 5] = 1 - .8
print(f'Profile sum: {prof.sum(axis=1)}')
print(f'Expected reward: {(rewards * ((prof.T @ prof) / (prof.T @ prof).sum())[:, :, None]).sum(axis=tuple((i for i in range(n_makers))))}')
print(f' --> {gtu.is_ne(scaled_rewards, prof, verbose=True)}')
print('--------------------------------------------')